<a href="https://colab.research.google.com/github/Bhavadharani275/Mini_Project_5/blob/main/Solar_Panels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load the dataset

In [ ]:
import os

dataset_path = '/content/drive/MyDrive/Faulty_solar_panel'
categories = ['Bird-drop', 'Clean', 'Dusty', 'Electrical-damage', 'Physical-Damage', 'Snow-Covered']

In [ ]:
import os
import numpy as np

dataset_dir = "/content/drive/MyDrive/Faulty_solar_panel"

# Get sorted list of folder names (important: keep consistent with training order)
classes = sorted(os.listdir(dataset_dir))
print("Detected Classes:", classes)

# Save to file
np.save("classes.npy", classes)
print("✅ classes.npy created successfully!")


# EDA

## Class Distribution

In [ ]:
import os
import pandas as pd
import plotly.express as px

# Path to your dataset (update this to match your Google Drive mount path)
dataset_dir = '/content/drive/MyDrive/Faulty_solar_panel'

# Count images in each class folder
class_counts = {cls: len(os.listdir(os.path.join(dataset_dir, cls)))
                for cls in os.listdir(dataset_dir)
                if os.path.isdir(os.path.join(dataset_dir, cls))}

# Convert to DataFrame
df = pd.DataFrame(list(class_counts.items()), columns=['Class', 'Count'])
df = df.sort_values(by='Count', ascending=False)

# Plotly bar chart
fig = px.bar(
    df,
    x='Class',
    y='Count',
    color='Class',
    text='Count',
    title="📊 Class Distribution of Solar Panel Images",
    color_discrete_sequence=px.colors.qualitative.Vivid
)

fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis_title="Defect Type",
    yaxis_title="Number of Images",
    showlegend=False,
    template="plotly_white"
)

fig.show()


## Visual Inspection of Image Samples

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

classes = list(class_counts.keys())
plt.figure(figsize=(12, 8))

for i, cls in enumerate(classes):
    folder = os.path.join(dataset_dir, cls)
    sample_img = random.choice(os.listdir(folder))
    img_path = os.path.join(folder, sample_img)
    img = mpimg.imread(img_path)

    plt.subplot(2, 3, i+1)
    plt.imshow(img)
    plt.title(cls)
    plt.axis('off')

plt.suptitle("🔍 Sample Images from Each Solar Panel Defect Category", fontsize=14)
plt.show()


## Image Count & File Insigh

In [ ]:
import os
from collections import Counter

dataset_dir = '/content/drive/MyDrive/Faulty_solar_panel'

# Count images and extensions
image_files = []
for cls in os.listdir(dataset_dir):
    folder = os.path.join(dataset_dir, cls)
    if os.path.isdir(folder):
        for f in os.listdir(folder):
            image_files.append(os.path.splitext(f)[1].lower())

Counter(image_files)


## Average Brightness per Class

In [ ]:
from PIL import Image
import numpy as np
import plotly.express as px

brightness_data = []

for cls in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, cls)
    if os.path.isdir(class_path):
        for img_name in os.listdir(class_path)[:30]:
            img_path = os.path.join(class_path, img_name)
            try:
                img = Image.open(img_path).convert('L')
                brightness = np.array(img).mean()
                brightness_data.append({'Class': cls, 'Brightness': brightness})
            except:
                continue

brightness_df = pd.DataFrame(brightness_data)
fig = px.box(
    brightness_df,
    x='Class',
    y='Brightness',
    color='Class',
    title="💡 Brightness Variation by Class",
    template="plotly_white"
)
fig.show()


## Correlation Heatmap of RGB Averages

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import plotly.figure_factory as ff

# 🔹 Extract average RGB values from all images
data = []
for cls in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, cls)
    if os.path.isdir(class_path):
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            try:
                img = Image.open(img_path).convert('RGB').resize((224, 224))
                arr = np.array(img)
                r, g, b = np.mean(arr[:, :, 0]), np.mean(arr[:, :, 1]), np.mean(arr[:, :, 2])
                data.append([cls, r, g, b])
            except:
                continue

# 🧮 Create DataFrame
color_df = pd.DataFrame(data, columns=['Class', 'Red', 'Green', 'Blue'])

# 🎨 Plotly Correlation Heatmap
corr = color_df[['Red', 'Green', 'Blue']].corr()
z = corr.values
fig = ff.create_annotated_heatmap(
    z=z,
    x=corr.columns.tolist(),
    y=corr.columns.tolist(),
    annotation_text=np.round(z, 2).astype(str),
    colorscale='Blues',
)
fig.update_layout(title="🔥 RGB Channel Correlation Heatmap", template="plotly_white")
fig.show()


# Data Preprocessing

## Load Images and Labels / Resize Images

In [ ]:
import cv2
import numpy as np

data = []
labels = []

for category in categories:
    folder_path = os.path.join(dataset_path, category)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        image = cv2.imread(img_path)
        if image is not None:
            # Resize image to 224x224 for CNN
            image = cv2.resize(image, (224, 224))
            data.append(image)
            labels.append(category)

data = np.array(data)
labels = np.array(labels)
print("Loaded images:", data.shape)


## Normalize Pixel Values

In [ ]:
data = data / 255.0  # Scale pixels to 0-1


## Encode Labels

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)


## Split Dataset

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(data, labels_categorical, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)  # ≈10% test


# Model Training

## Common Setup

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np

# ✅ Image Augmentation
datagen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X_train)

num_classes = y_train.shape[1]


## ResNet50

In [ ]:
from keras.applications import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Unfreeze last 100 layers
for layer in base_model.layers[:-100]:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(num_classes, activation='softmax')(x)

resnet_model = Model(inputs=base_model.input, outputs=output)

resnet_model.compile(optimizer=Adam(learning_rate=1e-4),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

checkpoint_resnet = ModelCheckpoint('SolarGuard_ResNet50.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)

history_resnet = resnet_model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=30,
    callbacks=[early_stop, checkpoint_resnet],
    verbose=1
)


## EfficientNetB0

In [ ]:
from keras.applications import EfficientNetB0

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers[:-80]:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(num_classes, activation='softmax')(x)

eff_model = Model(inputs=base_model.input, outputs=output)

eff_model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

checkpoint_eff = ModelCheckpoint('SolarGuard_EfficientNetB0.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

history_eff = eff_model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=30,
    callbacks=[early_stop, checkpoint_eff],
    verbose=1
)


## MobileNetV2

In [ ]:
from keras.applications import MobileNetV2

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers[:-70]:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(num_classes, activation='softmax')(x)

mobile_model = Model(inputs=base_model.input, outputs=output)

mobile_model.compile(optimizer=Adam(learning_rate=1e-4),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

checkpoint_mob = ModelCheckpoint('SolarGuard_MobileNetV2.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

history_mob = mobile_model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=30,
    callbacks=[early_stop, checkpoint_mob],
    verbose=1
)


# Model Evaluation

## Evaluation Metrics Code (for all models)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate_model(model, X_test, y_test, model_name="Model"):
    # Predict probabilities
    y_pred_probs = model.predict(X_test)
    # Convert to class labels
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Compute metrics
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    print(f"\n📊 Evaluation Results for {model_name}:")
    print(f"✅ Accuracy: {acc:.4f}")
    print(f"🎯 Precision: {prec:.4f}")
    print(f"📈 Recall: {rec:.4f}")
    print(f"⚖️ F1-score: {f1:.4f}")

    print("\n📋 Classification Report:")
    print(classification_report(y_true, y_pred))

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"Confusion Matrix - {model_name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

    return acc, prec, rec, f1


## Run Evaluation for Each Model

In [ ]:
# ResNet50
acc_resnet, prec_resnet, rec_resnet, f1_resnet = evaluate_model(resnet_model, X_test, y_test, "ResNet50")

In [ ]:
# EfficientNetB0
acc_eff, prec_eff, rec_eff, f1_eff = evaluate_model(eff_model, X_test, y_test, "EfficientNetB0")

In [ ]:
# MobileNetV2
acc_mob, prec_mob, rec_mob, f1_mob = evaluate_model(mobile_model, X_test, y_test, "MobileNetV2")

## Compare Metrics Across Models

In [ ]:
import pandas as pd

results_df = pd.DataFrame({
    "Model": ["ResNet50", "EfficientNetB0", "MobileNetV2"],
    "Accuracy": [acc_resnet, acc_eff, acc_mob],
    "Precision": [prec_resnet, prec_eff, prec_mob],
    "Recall": [rec_resnet, rec_eff, rec_mob],
    "F1-Score": [f1_resnet, f1_eff, f1_mob]
})

print("\n🔎 Model Comparison:")
print(results_df.sort_values(by="Accuracy", ascending=False).to_string(index=False))


## Visualization of Performance

In [ ]:
import pandas as pd

results_df = pd.DataFrame({
    "Model": ["ResNet50", "EfficientNetB0", "MobileNetV2"],
    "Accuracy": [acc_resnet, acc_eff, acc_mob],
    "Precision": [prec_resnet, prec_eff, prec_mob],
    "Recall": [rec_resnet, rec_eff, rec_mob],
    "F1-Score": [f1_resnet, f1_eff, f1_mob]
})


In [ ]:
import plotly.express as px

fig = px.bar(
    results_df.melt(id_vars="Model", var_name="Metric", value_name="Score"),
    x="Model",
    y="Score",
    color="Metric",
    barmode="group",
    text="Score",
    title="📊 SolarGuard Model Performance Comparison",
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig.update_layout(
    title_font=dict(size=22, family='Arial', color='darkblue'),
    xaxis_title="Deep Learning Model",
    yaxis_title="Metric Value",
    yaxis=dict(range=[0, 1]),
    legend_title_text="Metrics",
    template="plotly_white",
    bargap=0.25
)

fig.show()


# Streamlit app

In [ ]:
# %%writefile app.py

In [ ]:
code= """
import streamlit as st
import numpy as np
from PIL import Image
import io
import os
import plotly.express as px
from keras.models import load_model

# -------- CONFIG --------
MODELS = {
    "ResNet50": "SolarGuard_ResNet50.keras",
    "EfficientNetB0": "SolarGuard_EfficientNetB0.keras",
    "MobileNetV2": "SolarGuard_MobileNetV2.keras"
}
CLASSES_NPY = "classes.npy"
IMG_SIZE = (224, 224)
# ------------------------

st.set_page_config(page_title="SolarGuard", layout="centered", page_icon="🌞")
st.title("🌞 SolarGuard — Solar Panel Defect Classification")
st.markdown("Upload an image and compare predictions from multiple deep learning models.")

# ---------------- UTILITIES ----------------
@st.cache_resource
def load_model_cached(path):
    return load_model(path)

@st.cache_resource
def load_classes():
    if os.path.exists(CLASSES_NPY):
        return np.load(CLASSES_NPY, allow_pickle=True).tolist()
    return ['Bird-drop', 'Clean', 'Dusty', 'Electrical-damage', 'Physical-Damage', 'Snow-Covered']

def preprocess_image(image: Image.Image):
    img = image.convert("RGB").resize(IMG_SIZE)
    arr = np.array(img).astype("float32") / 255.0
    return np.expand_dims(arr, axis=0)

def maintenance_recommendation(pred_class):
    recs = {
        "Bird-drop": ("Clean locally; consider bird deterrents.", "Medium"),
        "Clean": ("No immediate action.", "Low"),
        "Dusty": ("Schedule cleaning or increase maintenance frequency.", "Medium"),
        "Electrical-damage": ("Immediate technical inspection required.", "High"),
        "Physical-Damage": ("Replace or repair panel (cracks/shattered glass).", "High"),
        "Snow-Covered": ("Remove snow manually or via tilt/heating.", "Medium")
    }
    return recs.get(pred_class, ("Inspect manually and verify.", "Medium"))
# -------------------------------------------

# Sidebar: model selection
with st.sidebar:
    st.header("⚙️ Model Settings")
    selected_models = st.multiselect(
        "Select models to compare:",
        list(MODELS.keys()),
        default=["ResNet50"]
    )
    st.markdown("---")
    st.caption("You can select one or multiple models for comparison.")

classes = load_classes()

# Main Upload Area
st.subheader("📤 Upload Solar Panel Image")
uploaded_file = st.file_uploader("Upload an image...", type=["jpg", "jpeg", "png"])

if uploaded_file:
    image = Image.open(io.BytesIO(uploaded_file.read()))
    st.image(image, caption="Uploaded Image", use_container_width=True)

    img_batch = preprocess_image(image)

    all_results = []
    with st.spinner("Predicting..."):
        for model_name in selected_models:
            model_path = MODELS[model_name]
            if not os.path.exists(model_path):
                st.error(f"Model file not found: {model_path}")
                continue
            model = load_model_cached(model_path)
            probs = model.predict(img_batch)[0]
            pred_idx = int(np.argmax(probs))
            pred_class = classes[pred_idx]
            conf = probs[pred_idx]
            all_results.append({
                "Model": model_name,
                "Predicted Class": pred_class,
                "Confidence": float(conf),
                "Probabilities": probs
            })

    # Show results
    for result in all_results:
        model_name = result["Model"]
        pred_class = result["Predicted Class"]
        conf = result["Confidence"]
        probs = result["Probabilities"]

        st.markdown(f"### 🧠 {model_name} Prediction")
        st.write(f"**Predicted Class:** `{pred_class}` — **Confidence:** {conf:.2%}")

        fig = px.bar(
            x=classes, y=probs,
            labels={'x': 'Class', 'y': 'Probability'},
            title=f"📊 {model_name} Prediction Probabilities",
            color=classes,
            color_discrete_sequence=px.colors.qualitative.Safe
        )
        fig.update_layout(yaxis=dict(range=[0,1]), template="plotly_white", showlegend=False)
        st.plotly_chart(fig, use_container_width=True)

        # Add recommendation
        rec_text, priority = maintenance_recommendation(pred_class)
        st.write(f"**Maintenance Priority:** {priority}")
        st.info(rec_text)

else:
    st.info("Please upload an image to start model comparison.")

# Footer
st.markdown("---")
st.caption("SolarGuard © — Multi-model solar panel defect analysis (ResNet50, EfficientNetB0, MobileNetV2).")

"""
with open("app.py", "w") as f:
    f.write(code)

In [ ]:
!pip install streamlit pyngrok plotly

# Run

In [ ]:
# Need to get ngrok authtoken to run the streamlit app in local so get the token form the ngrok site

from pyngrok import ngrok

# Open a tunnel to port 8501
public_url = ngrok.connect(8501)
print(public_url)

# Run your Streamlit app
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0